# Code Constructions

Build HGP, QLP, and BPC codes and verify their CSS logicals.


### Hypergraph product (HGP) code parameters

- `h`: parity check matrix of the classical LDPC code (loaded from `../parity_check_matrices`)
- `HgpCode(h, h)`: constructs the HGP code from two classical parity check matrices
- `seed`: used by `build_graph` to assign edge directions and colors


In [1]:
import numpy as np
import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


In [2]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_graph(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 9, 'pairing_is_identity': True, 'ok': True}


### Lifted product (QLP) code parameters

- `b`: base matrix of monomial powers for the polynomial entries
- `lift_size`: circulant size used to lift each monomial entry
- `QlpCode(b, b, lift_size)`: builds a QLP code from two base matrices


In [3]:
# Build a small QLP code
lift_size = 16
b = np.array(
    [
        [0, 0, 0, 0, 0],
        [0, 2, 4, 7, 11],
        [0, 3, 10, 14, 15],
    ]
)
code = QlpCode(b, b, lift_size)
code.build_graph(seed=1)
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 232, 'rank_hx': 232, 'rank_lz': 80, 'rank_lx': 80, 'k_expected': 80, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 312, 'rank_hx_plus_lx': 312, 'dim_ker_hz': 312, 'dim_ker_hx': 312, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 80, 'pairing_is_identity': True, 'ok': True}


### Balanced product cyclic (BPC) code parameters

- `p1`, `p2`: polynomial powers defining the two circulant blocks
- `lift_size`: circulant size of each block
- `factor`: size of the cyclic subgroup factored out

To match arXiv:2411.03302, use `p2` as the transpose convention: each entry is `lift_size` minus the powers listed in the paper.


In [4]:
# Build a small BPC code
lift_size, factor = 15, 3
p1 = [0, 1, 5]
p2 = [0, 8, 13]
# p2 uses lift_size minus values compared to arXiv:2411.03302 (transpose convention).
code = BpcCode(p1, p2, lift_size, factor)
code.build_graph(seed=1)
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 41, 'rank_hx': 41, 'rank_lz': 8, 'rank_lx': 8, 'k_expected': 8, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 49, 'rank_hx_plus_lx': 49, 'dim_ker_hz': 49, 'dim_ker_hx': 49, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 8, 'pairing_is_identity': True, 'ok': True}
